In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from Learners import *

### Data

In [5]:
import pandas as pd
df=pd.read_csv('../data.csv',index_col=False)
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean
0,male,1941,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
1,female,1947,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
2,male,1951,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
3,female,1950,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
4,female,1982,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143


In [6]:
df.loc[df['sex']=='male', 'sex']=0
df.loc[df['sex']=='female', 'sex']=1
df['age']=2019-df['yob']
for item in ['g2000','g2002','p2000','p2002','p2004','voted']:
    df.loc[df[item]=='yes', item]=1
    df.loc[df[item]=='Yes', item]=1
    df.loc[df[item]=='no', item]=0
    df.loc[df[item]=='No', item]=0

In [7]:
df = df[(df['treatment']==' Control') | (df['treatment']==' Neighbors')]
df.loc[df['treatment']==' Control', 'treatment']=0
df.loc[df['treatment']==' Neighbors', 'treatment']=1

In [8]:
df.head()

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,age
5,0,1981,0,0,yes,0,0,0,0,1,0,3.0,3,21,0.047619,0.904762,38
6,1,1959,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,60
7,0,1956,1,1,yes,0,1,0,0,1,1,3.0,3,21,0.047619,0.904762,63
8,1,1968,0,0,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,51
9,0,1967,1,1,yes,0,1,0,0,1,0,4.0,2,21,0.047619,0.904762,52


In [9]:
X=np.array(df[['sex','age','g2000','g2002','p2000','p2002','p2004']], dtype=np.int)
y=np.array(df['voted'],dtype=np.int)
treatment=np.array(df['treatment'],dtype=np.int)

In [10]:
print(X[treatment==1].shape,X[treatment==0].shape)

(38201, 7) (191243, 7)


### Honest RF

In [11]:
from econml.grf import RegressionForest

In [12]:
# T
learner_t = BaseTClassifier(learner0=RegressionForest(), learner1=RegressionForest(), type_=1)
cate_t = learner_t.get_cate(X, y, treatment)
cate_t=cate_t.reshape(-1)
# S
learner_s = BaseSClassifier(learner=RegressionForest(), type_=1)
cate_s = learner_s.get_cate(X, y, treatment)
cate_s=cate_s.reshape(-1)

In [ ]:
# X
learner_x = BaseXClassifier(learner=RegressionForest(), cate_learner=RegressionForest(), prospensity_learner=RegressionForest(),type_=1)
cate_x = learner_x.get_cate(X,y,treatment,p=0.4)
cate_x=cate_x.reshape(-1)

In [10]:
np.corrcoef(cate_t, cate_s)

array([[1.        , 0.92786332],
       [0.92786332, 1.        ]])

In [11]:
cate_t

array([0.06856818, 0.13311989, 0.03781513, ..., 0.05188794, 0.01830803,
       0.0868814 ])

In [12]:
cate_s

array([0.07198149, 0.13006543, 0.07689693, ..., 0.05299774, 0.03228469,
       0.08701159])

In [ ]:
cate_x

### RF

In [9]:
# T
learner_t = BaseTClassifier(learner0=RandomForestClassifier(), learner1=RandomForestClassifier())
cate_t = learner_t.get_cate(X, y, treatment)
# S
learner_s = BaseSClassifier(learner=RandomForestClassifier())
cate_s = learner_s.get_cate(X, y, treatment)
# X

In [14]:
learner_x = BaseXClassifier(learner=RandomForestClassifier(), cate_learner=RandomForestRegressor(), prospensity_learner=RandomForestClassifier())
cate_x = learner_x.get_cate(X, y, treatment)

In [15]:
cate_x

array([ 0.87017477,  0.30309826,  0.3330472 , ...,  0.65844311,
       -0.57013709,  0.29968427])

In [10]:
cate_t

array([0.06949067, 0.15446101, 0.05590717, ..., 0.10598052, 0.06348216,
       0.04662858])

In [11]:
cate_s

array([0.06649312, 0.15259589, 0.06178408, ..., 0.08760933, 0.04656123,
       0.06462179])

In [12]:
np.corrcoef(cate_t, cate_s)

array([[1.        , 0.98916606],
       [0.98916606, 1.        ]])